In [1]:
import pandas as pd
import numpy as np

In [ ]:
review = pd.read_csv('(완)항목별리뷰데이터.csv').iloc[:,1:]
review['항목'] = review['항목'].fillna('기타')

In [30]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317390 entries, 0 to 317389
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   회사명     317390 non-null  object
 1   플랫폼     317390 non-null  object
 2   항목      317390 non-null  object
 3   의견      317390 non-null  object
 4   리뷰      317390 non-null  object
dtypes: object(5)
memory usage: 12.1+ MB


In [33]:
# 리뷰 텍스트 형식 정리

name = []
category = []
opinion = []
texts = []

# 긍정/부정 리뷰 각각 수집
for n in review['회사명'].unique():
    for c in review['항목'].unique():
        for p in review['의견'].unique():
            tmp = review[(review['회사명'] == n) & (review['항목'] == c) & (review['의견'] == p)]
            name.append(n)
            category.append(c)
            opinion.append(p)
            texts.append(tmp['리뷰'].tolist())

# 긍정/부정 통합 수집
for n in review['회사명'].unique():
    for c in review['항목'].unique():
        tmp = review[(review['회사명'] == n) & (review['항목'] == c) & (review['의견'] == p)]
        name.append(n)
        category.append(c)
        opinion.append('통합')
        texts.append(tmp['리뷰'].tolist())


total_data = pd.DataFrame()
total_data['회사명'] = pd.Series(name)
total_data['항목'] = pd.Series(category)
total_data['의견'] = pd.Series(opinion)
total_data['리뷰'] = pd.Series(texts)
total_data
total_data.to_excel("최종_리뷰텍스트통합.xlsx" ,index=True)

In [34]:
total_data

,회사명,항목,의견,리뷰
0,(주)위메프,연봉,장점,[역세권 사무실워라밸 좋음구내식당(주로 돈가스 같은 저렴한 재료만씀)자사몰 이용시 ...
1,(주)위메프,연봉,단점,[회사매출 곤두박질침연봉동결 복지 다없앰경영진의 무능함우당탕탕 일이 생기고 없어지는...
2,(주)위메프,조직문화,장점,"[7시 칼퇴근수평적 분위기맛있는 구내식당 (점심:2500원, 저녁:2000원), 수..."
3,(주)위메프,조직문화,단점,"[이커머스 시장 자체가 전반적으로 실적이 좋치 못하고,위메프도 마찬가지라서 경영에 ..."
4,(주)위메프,기타,장점,"[정시퇴근뿐이지만 그안에 모든걸 다 해결해야 퇴근한다….., 자유로운 연차 사용, ..."
...,...,...,...,...
14935,한양사이버대학교,연봉,통합,"[계약직 급여가 너무 짜다. 아르바이트 최저시급과 비슷하다., 고인물(아줌마)들이 ..."
14936,한양사이버대학교,조직문화,통합,"[고용불안정(계약직이많음), 복잡한 행정 절차가 많음, 승진이없음]"
14937,한양사이버대학교,기타,통합,"[각각의 사업단 센터로 기간제기 때문에 어딜 들어가느냐가 중요합니다, 좋은분도 있고..."
14938,한양사이버대학교,워라밸,통합,"[지방 사립대, 회식, 야근수당 상한선 20, 성희롱, 교직원 하대, 까다로운 행정..."


## 키워드 추출

In [35]:
from konlpy.tag import Mecab 
mecab = Mecab(dicpath='C:/mecab/mecab-ko-dic')

In [36]:
tag_classes = ['NNG', 'NNP','VA', 'VV+EC','VV+ETM', 'NP', 'NNBC', 'VA+EC','SL']

def mecab_tokenizer(text):
    result = mecab.pos(text, join = True)
    clean_word = [w for w in result if tag_classes[0] in w or
                  tag_classes[1] in w or tag_classes[2] in w or tag_classes[3] in w or tag_classes[4] in w or
                  tag_classes[5] in w or tag_classes[6] in w or tag_classes[7] in w or tag_classes[0] in w]
    return clean_word

In [54]:
# 리뷰
stopword = ['가능','강도','결정','경영','경우','관리','구조','그렇','근무','기관','기본','기업','대비','대한','따라','바이','비해','정도','따라','이상','위한','해야','해도','여기', '자기','매출','해서','코로나','나옴','인해','포인트','관련','최근','중요','위해','업계','부분','느낌','본인','다르','다닐','자체','기준','특성','처리','그룹','전체','인한','어디','의사','제외','가지','상황','분야','대해','이름','방식','자신','누구','그런지','그것','직군','우리','일반','특정','와서','거기','구성','내용','다음','가진','기타','인지','통해','대해서','형태','단위','이것저것','당시']

In [56]:
from textrank import KeywordSummarizer

summarizer = KeywordSummarizer(tokenize=mecab_tokenizer,
                               min_count=2, 
                               min_cooccurrence=1)

In [62]:
# 키워드 추출

name = []
category = []
opinion = []
word = []
score = []
index = []

for i in len(total_data):
    try:
        summary = summarizer.summarize(total_data['리뷰'][i], topk=20)
        for nth in range(len(summary)):
            name.append(total_data.iloc[i,]['회사명'])
            category.append(total_data.iloc[i,]['항목'])
            opinion.append(total_data.iloc[i,]['의견'])
            word.append(summary[nth][0].split('/')[0])
            score.append(summary[nth][1])
            index.append(nth+1)
    except:
        pass
    
    
final = pd.DataFrame()
final['회사명'] = pd.Series(name)
final['항목'] = pd.Series(category)
final['의견'] = pd.Series(opinion)
final['단어'] = pd.Series(word)
final['빈도'] = pd.Series(score)
final['키워드인덱스'] = pd.Series(index)

final.to_excel('항목별워드클라우드_최종.xlsx', index = True)